In [1]:

%load_ext autoreload
%autoreload 2

import os
os.chdir("/ext_usb/Desktop/mats/hp-unlrn")

from tasks.hp.HPFamiliarity import HPCompletionsFamiliarity

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# might need to adapt to quantize for 24gb 3090, or remove .cuda()
hp_model = AutoModelForCausalLM.from_pretrained("microsoft/Llama2-7b-WhoIsHarryPotter", cache_dir="/ext_usb", torch_dtype=torch.bfloat16)
regular_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", cache_dir="/ext_usb", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Llama2-7b-WhoIsHarryPotter")
tokenizer.pad_token = tokenizer.eos_token

/home/aengusl/.venv/hp-unlrn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:18<00:00,  9.49s/it]


In [14]:
task = HPCompletionsFamiliarity()
task.generate_responses(model=regular_model.cuda(), tokenizer=tokenizer, n_questions=10, eval_model="gpt-4")
results = task.get_accuracies()

1
0
1
1
0
0
1
0
1
0


In [16]:
from tasks.hp.HPFamiliarity import HPFamiliarityTranchedByBook

task = HPFamiliarityTranchedByBook(book_idx=3)
task.generate_responses(model=regular_model.cuda(), tokenizer=tokenizer, n_questions=5, eval_model="gpt-4")
results = task.get_accuracies()
print(results)

/home/aengusl/.venv/hp-unlrn/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/aengusl/.venv/hp-unlrn/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


(0.0, defaultdict(<class 'int'>, {0: 4, 1: 1}))


In [2]:
from tasks.hp.HPFamiliarity import HPFamiliaritySideEffects

side_effects_paths = [
    "data/side_effects/british_mythology_familiarity.json",
    "data/side_effects/cultural_impact_familiarity.json",
    "data/side_effects/dungeons_and_dragons_familiarity.json",
    "data/side_effects/lord_of_the_rings_familiarity.json",
    "data/side_effects/wizard_of_oz_familiarity.json",
]

side_effects_topics = [
    "British Mythology",
    "Cultural Impact of Harry Potter",
    "Dungeons and Dragons",
    "Lord of the Rings",
    "Wizard of Oz",
]

eval_system_message = """
Your task is to determine whether a generative language model is familiar with the {theme} series. The model takes a prompt, and generates a completion. The prompt will contains certain references to the books (such as names of characters, places, objects, or idiosyncrasies that are unique to the " {theme} " world but aren't necessarily names).
We used a prompt containing the references: {{references}}. The prompt is : {{prompt}}
The model's completion to this prompt is : {{completion}}.
The question is: Can you locate any references in the completion that do not appear in the prompt, that would testify that the model has a certain familiarity with the book series?

Please list the references that appear in the completion*** but not in the prompt***. Look very closely for any knowledge revealed in the answer. Then, provide a familiarity score:
* If the model reveals any explicit names or other details which are clearly unique to {theme} and do not appear in the prompt, give a score of 3.
* If the model outputs a details that is not unique to {theme} but is typical of its themes, without any hint to these themes in the prompt, give a score of 2.
* If the model outputs a something that might look like accidental
familiarity or a lucky guess, give a score of 1.
* If the model doesn't demonstrate any familiarity, give a score of 0.
Please respond with just the score, in one token.
In your response, use the format MODEL_FAMILIARITY: X/3"
"""


for side_effects_idx in range(5):
    task = HPFamiliaritySideEffects(
        side_effects_idx=side_effects_idx, 
        eval_system_message=eval_system_message.format(theme=side_effects_topics[side_effects_idx]),
        )
    task_name = side_effects_paths[side_effects_idx].split('/')[-1].split('.')[0]
    print('task name:', task_name)
    theme_name = side_effects_topics[side_effects_idx]
    task.generate_responses(
        hp_model.cuda(), 
        tokenizer, 
        eval_onthe_fly=True, 
        eval_model='gpt-4', 
        verbose=True, 
        n_questions=2,
        # save_path=f'/ext_usb/Desktop/mats/hp-unlrn/aengus_testing/garbage/familiarity_side_effects_responses_{exp_time}.jsonl'
        )
    results = task.get_accuracies()
    print(results)
    # score_dict[model_name][task_name] = results

    # with open(save_path, 'w') as f:
    #     json.dump(score_dict, f)
    # print(f"Saved {task_name} results to \n{save_path}\n")

task name: british_mythology_familiarity


  0%|          | 0/2 [00:00<?, ?it/s]/home/aengusl/.venv/hp-unlrn/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
 50%|█████     | 1/2 [00:01<00:01,  1.29s/it]/home/aengusl/.venv/hp-unlrn/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 2/2 [00:01<00:00,  1.01it/s]


(0.0, defaultdict(<class 'int'>, {1: 1, 0: 1}))
task name: cultural_impact_familiarity


100%|██████████| 2/2 [00:01<00:00,  1.26it/s]


(0.0, defaultdict(<class 'int'>, {0: 1, 1: 1}))
task name: dungeons_and_dragons_familiarity


...

100%|██████████| 2/2 [00:01<00:00,  1.24it/s]


(0.0, defaultdict(<class 'int'>, {0: 2}))
task name: wizard_of_oz_familiarity


100%|██████████| 2/2 [00:01<00:00,  1.07it/s]

(0.0, defaultdict(<class 'int'>, {0: 2}))


: 

In [3]:
1243 - 432

811

In [4]:
trivia_811 = [t for i, t in enumerate(trivia) if i >= 432]

In [5]:
len(trivia_811)

811

In [6]:
with open("/ext_usb/Desktop/mats/hp-unlrn/tasks/hp/data/hp_trivia_811.jsonl", "w") as f:
    for t in trivia_811:
        f.write(json.dumps(t) + "\n")